In [3]:
import numpy as np
import itertools
from scipy.signal import detrend
import os
import pandas as pd

In [4]:
def calculate_hurst_exponent(time_series, nvals):
    N = len(time_series)
    F = np.zeros(len(nvals), dtype=float)

    for i, n in enumerate(nvals):
        if n > N:
            continue
        segments = np.array_split(np.arange(N), np.arange(n, N+1, n))
        rms_fluctuations = np.zeros(len(segments), dtype=float)

        for j, segment in enumerate(segments):
            segment_data = time_series[segment]
            detrended_segment = detrend(segment_data)
            rms_fluctuations[j] = np.sqrt(np.mean(detrended_segment**2))

        F[i] = np.mean(rms_fluctuations)

    coeffs = np.polyfit(np.log(nvals), np.log(F), 1)
    hurst_exponent = coeffs[0]
    return hurst_exponent

In [5]:
def generate_partitions(Q):
    if Q == 4:
        partitions = [( (1, 2), (3, 4) ), ( (1, 3), (2, 4) ), ( (1, 4), (2, 3) )]
    elif Q == 5:
        partitions = [( (1, 2), (3, 4, 5) ), ( (1, 3), (2, 4, 5) ), ( (1, 4), (2, 3, 5) ),
                      ( (1, 5), (2, 3, 4) ), ( (2, 3), (1, 4, 5) ), ( (2, 4), (1, 3, 5) ),
                      ( (2, 5), (1, 3, 4) ), ( (3, 4), (1, 2, 5) ), ( (3, 5), (1, 2, 4) ),
                      ( (4, 5), (1, 2, 3) )]
    else:
        raise ValueError("Only Q=4 or Q=5 is supported for generating partitions.")
    
    return partitions

In [6]:
def calculate_hurst_for_partitions(microstate_sequence, nvals, Q):
    hurst_values = []
    partitions = generate_partitions(Q)

    for partition in partitions:
        group1, group2 = partition
        partitioned_sequence = np.where(np.isin(microstate_sequence, group1), 1, -1)
        
        # Calculate Hurst exponent for this partition
        hurst = calculate_hurst_exponent(partitioned_sequence, nvals)
        hurst_values.append(hurst)

    # Calculate the average Hurst exponent across all partitions
    average_hurst = np.mean(hurst_values)
    return average_hurst

In [7]:
# 定义读取文件和计算Hurst指数的函数
def calculate_hurst_for_subject(file_path, nvals, Q):
    # 读取微状态序列
    microstate_sequence = np.loadtxt(file_path)
     # 提取前1000个数据点
    microstate_sequence = microstate_sequence[:1976]
    # 计算Hurst指数
    hurst = calculate_hurst_for_partitions(microstate_sequence, nvals, Q)
    return hurst

In [12]:
# 设置文件路径和参数
file_path = r'F:\EEGmicrostate\microstate_2_20\groupMicrostate\EO\MicorstateSeuqnce2group\all\full'
nvals = np.logspace(np.log10(50), np.log10(3000), 100).astype(int)  # 时间尺度
Q = 5  # 假设有5个微状态


In [13]:
# 遍历文件夹中的所有文件
for file_name in os.listdir(file_path):
    if file_name.endswith(".txt"):
        file_full_path = os.path.join(file_path, file_name)
        hurst = calculate_hurst_for_subject(file_full_path, nvals, Q)
        print(f"File: {file_name}, Hurst exponent: {hurst}")

ValueError: cannot reshape array of size 0 into shape (0,newaxis)